<h1>Differential Metabolites</h1>

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
basedir = '../../'
sys.path.append(basedir)

from collections import Counter
import csv
import math

import numpy as np
import pandas as pd
import networkx as nx
import pylab as plt

import matplotlib as mpl
import matplotlib.cm as cm

from IPython.display import display
from lda_for_fragments import Ms2Lda
from visualisation.networkx.lda_visualisation import *

# get rid of annoying warnings
import warnings
warnings.filterwarnings('ignore')

<h1>Define analysis method</h1>

In [ ]:
def run_analysis(project_file, annotation_file, pimp_file):
    
    # load m2lda object and do thresholding
    ms2lda = Ms2Lda.resume_from(project_file)
    ms2lda.do_thresholding(th_doc_topic=0.05, th_topic_word=0.01)

    # read the annotation file
    print
    print "Annotation file"
    motif_annotation = {}
    motif_idx = {}
    i = 0
    for item in csv.reader(open(annotation_file), skipinitialspace=True):
        key = int(item[0])
        val = item[1]
        print str(key) + "\t" + val
        motif_annotation[key] = val
        motif_idx[key] = i
        i += 1

    motifs_of_interest = motif_annotation.keys()    
    norm = mpl.colors.Normalize(vmin=min(motif_idx.values()), vmax=max(motif_idx.values()))
    cmap = cm.gist_rainbow
    motif_colour = cm.ScalarMappable(norm=norm, cmap=cmap)    
    
    # get the network graph out for the motifs of interest
    G = get_network_graph(ms2lda, motifs_of_interest)
    print "\n" + nx.info(G)  
    print

    # print out some info
    ms1_count = 0
    nodes = G.nodes(data=True)
    for node_id, node_data in nodes:
        # 1 for doc, 2 for motif
        if node_data['group'] == 1: 
            ms1_count += 1
    print "%d (out of %d) MS1 peaks found in the graph" % (ms1_count, ms2lda.ms1.shape[0])    
    
    # load the pimp differential analysis file for matching
    de_peaks = []
    with open(pimp_file, "rb") as infile:
       reader = csv.reader(infile)
       next(reader, None)  # skip the headers
       for row in reader:
        PiMP_ID = int(row[0])
        polarity = row[1]
        mz = float(row[2])
        rt = float (row[3])
        mh_intensity = float(row[4])
        tup = (PiMP_ID, polarity, mz, rt, mh_intensity)
        de_peaks.append(tup)

    print
    print "PiMP list: "
    for tup in de_peaks:
        print tup
        
    # do the matching
    mass_tol = 3
    rt_tol = 12

    std = np.array(de_peaks)
    std_mz = np.array([x[2] for x in de_peaks])
    std_rt = np.array([x[3] for x in de_peaks])
    matches = {}

    ms1_label = {}
    for row in ms2lda.ms1.itertuples(index=True):
        peakid = row[1]
        mz = row[5]
        rt = row[4]

        # the following line is hacky for pos mode data
        mass_delta = mz*mass_tol*1e-6
        mass_start = mz-mass_delta
        mass_end = mz+mass_delta
        rt_start = rt-rt_tol
        rt_end = rt+rt_tol

        match_mass = (std_mz>mass_start) & (std_mz<mass_end)
        match_rt = (std_rt>rt_start) & (std_rt<rt_end)
        match = match_mass & match_rt

        res = std[match]
        if len(res) == 1:
            closest = tuple(res[0])
            matches[closest] = row
            ms1_label[row[1]] = closest[1]        
        elif len(res)>1:
            closest = None
            min_dist = sys.maxint
            for match_res in res:
                match_mz = float(match_res[2])
                match_rt = float(match_res[3])
                dist = math.sqrt((match_rt-rt)**2 + (match_mz-mz)**2)
                if dist < min_dist:
                    min_dist = dist
                    closest = match_res
            closest = tuple(closest)
            matches[closest] = row
            ms1_label[row[1]] = closest[1]

    print "Matches found %d/%d" % (len(matches), len(std))
    print

    ms1_list = []
    for match in matches:
        key = str(match)
        ms1_row = matches[match]
        value = str(ms1_row)
        pid = ms1_row[1]
        print "Standard %s" % key
        print "MS1 %s" % value
        print
        ms1_list.append(pid)
        
    # print the motifs and count their occurences
    m2m_list = motifs_of_interest
    word_map, motif_words = ms2lda.print_motif_features(quiet=True)

    c = Counter() # count the motif occurences
    for i in range(len(ms1_list)):

        ms1 = ms1_list[i]
        df = print_report(ms2lda, G, ms1, motif_annotation, motif_words, motif_colour, motif_idx, word_map, xlim_upper=770)
        if df is not None:

            # display(df) # show the table to see the mz, annotations, etc

            # get the motif ids in the dataframe
            fragment_motif_ids = df[['fragment_motif']].values.flatten()
            loss_motif_ids = df[['loss_motif']].values.flatten()

            # get rid of nan values
            fragment_motif_ids = fragment_motif_ids[~np.isnan(fragment_motif_ids)]
            loss_motif_ids = loss_motif_ids[~np.isnan(loss_motif_ids)]

            # store the unique counts
            combined = np.append(fragment_motif_ids, loss_motif_ids).astype(int)
            combined = set(combined.tolist())
            c.update(combined)
            
    return c

In [ ]:
def plot_counts(c):
    
    motifs = []
    counts = []
    for key, value in c.most_common(): # iterated in descending order
        if value > 1:
            motifs.append(key)
            counts.append(value)

    motifs = np.array(motifs)
    counts = np.array(counts)

    print motifs
    print counts
    
    ind = np.arange(len(motifs))
    width = 0.8
    fig, ax = plt.subplots()
    ax.bar(ind, counts, width, color='b')

    ax.set_ylabel('Counts')
    ax.set_xlabel('M2M')
    ax.set_title('Motif Counts')
    _ = ax.set_xticks(ind + width)
    _ = ax.set_xticklabels(motifs)
    labels = ax.get_xticklabels()
    _ = plt.setp(labels, rotation=90, fontsize=14)

<h2>Differential metabolites BeerQC (4) UP vs Beer3</h2>

Trying to find all the peaks in the differential metabolites (UP) list and plotting their motifs..

In [ ]:
project_file = 'results/Manuscript_BeerQCPOSmode_EFassigner_ALLextended.project'
annotation_file = 'results/beer4pos_annotation_Nov2015.csv'
pimp_file = 'results/DifferentialMetabolites_BeerQC.csv'

In [ ]:
c = run_analysis(project_file, annotation_file, pimp_file)

In [ ]:
plot_counts(c)

<h2>Differential metabolites Beer3 UP vs Beer QC</h2>

In [ ]:
project_file = 'results/Manuscript_Beer3POSmode_EFassigner_ALLextended.project'
annotation_file = 'results/beer3pos_annotation_Nov2015.csv'
pimp_file = 'results/DifferentialMetabolites_Beer3.csv'

In [ ]:
c = run_analysis(project_file, annotation_file, pimp_file)

In [ ]:
plot_counts(c)